# Advanced Querying Mongo

Importing libraries and setting up connection

In [74]:
from pymongo import MongoClient
from pymongo import ASCENDING, DESCENDING


In [136]:
#el nombre del db no coincide con companies porque lo he hecho a mano directamente importandolo desde Mongodb compass y le puse ese nombre
dbName = "Crunchbase_Dataset"
mongodbURL = f"mongodb://localhost/{dbName}"
client = MongoClient(mongodbURL, connectTimeoutMS=2000, serverSelectionTimeoutMS=2000)
db = client.get_database()

### 1. All the companies whose name match 'Babelgum'. Retrieve only their `name` field.

In [76]:
cursor = db.companies.find({'name':'Babelgum'}, {"name":1})
for e in cursor:
    print(e['name'])

Babelgum


### 2. All the companies that have more than 5000 employees. Limit the search to 20 companies and sort them by **number of employees**.

In [111]:
cursor = db.companies.find({"number_of_employees":{"$gt":5000}}).limit(10)
for e in cursor:
    print(e['name'])

Facebook
eBay
Cisco
Yahoo!
Google
Intel
Nintendo
Adobe Systems
Sony
PayPal


### 3. All the companies founded between 2000 and 2005, both years included. Retrieve only the `name` and `founded_year` fields.

In [78]:
years = list(range(2000, 2006))
cursor = db.companies.find({"founded_year":{"$in":years}}, {'name':1, 'founded_year': 1}).limit(10)
for e in cursor:
    print(e['name'], e['founded_year'])

Wetpaint 2005
Zoho 2005
Digg 2004
Facebook 2004
Omnidrive 2005
StumbleUpon 2002
Gizmoz 2003
Helio 2005
Plaxo 2002
Technorati 2002


### 4. All the companies that had a Valuation Amount of more than 100.000.000 and have been founded before 2010. Retrieve only the `name` and `ipo` fields.

In [110]:
cursor = db.companies.find({'$and':[{'ipo.valuation_amount':{'$gt': 100000000}},
                                    {'founded_year':{'$lt':2010}}]}, 
                           {'name':1,'ipo':1}).limit(3)
for e in (cursor):
    print(e['name'], e['ipo'])

Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Twitter {'valuation_amount': 18100000000, 'valuation_currency_code': 'USD', 'pub_year': 2013, 'pub_month': 11, 'pub_day': 7, 'stock_symbol': 'NYSE:TWTR'}
Yelp {'valuation_amount': 1300000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 3, 'pub_day': 2, 'stock_symbol': 'NYSE:YELP'}


### 5. All the companies that have less than 1000 employees and have been founded before 2005. Order them by the number of employees and limit the search to 10 companies.

In [81]:
cursor = db.companies.find({'$and':[{'number_of_employees':{'$lt': 1000}},
                                    {'founded_year':{'$lt':2005}}]}, 
                           {'number_of_employees':1,'name':1}).limit(10).sort([('number_of_employees',DESCENDING)])
for i in cursor:
    print(i['name'],i['number_of_employees'],)

Infinera Corporation 974
NorthPoint Communications Group 948
888 Holdings 931
Forrester Research 903
SonicWALL 900
Webmetrics 900
Cornerstone OnDemand 881
Yelp 800
MySpace 800
ZoomInfo 800


### 6. All the companies that don't include the `partners` field.

In [83]:
cursor = db.companies.find({'partners':{'$exists':'false'}}).limit(20)
for e in cursor:
    print(e['name'])

Wetpaint
AdventNet
Zoho
Digg
Facebook
Omnidrive
Postini
Geni
Flektor
Fox Interactive Media
Twitter
StumbleUpon
Gizmoz
Scribd
Slacker
Lala
Helio
eBay
MeetMoi
Joost


### 7. All the companies that have a null type of value on the `category_code` field.

In [85]:
curse = db.companies.find({'category_code': 'null'}).limit(10)
for i in curse:
    print(i['name'])

Collective
Snimmer
KoolIM
Level9 Media
VidKing
Drigg
SpaceTime
Touch Clarity
MMDAYS
Inside Group


### 8. All the companies that have at least 100 employees but less than 1000. Retrieve only the `name` and `number of employees` fields.

In [98]:
cur =db.companies.find(
    {'$and':[{'number_of_employees':{'$gt':100}},{'number_of_employees':{'$lt':1000}}]},
                               {'name':1,'number_of_employees':1}
).limit(10)
for i in cur:
    print(i['name'],i['number_of_employees'])


AdventNet 600
AddThis 120
OpenX 305
LifeLock 644
Jajah 110
Livestream 120
Ustream 250
iContact 300
Yelp 800
Dailymotion 120


### 9. Order all the companies by their IPO price in a descending order.

In [109]:
cur = db.companies.find({'ipo.valuation_amount': {'$exists': 'true'}}).sort([('ipo.valuation_amount',DESCENDING)]).limit(3)
for e in cur:
    print(e['name'],e['ipo'])

GREE {'valuation_amount': 108960000000, 'valuation_currency_code': 'JPY', 'pub_year': 2008, 'pub_month': 12, 'pub_day': 17, 'stock_symbol': '3632'}
Facebook {'valuation_amount': 104000000000, 'valuation_currency_code': 'USD', 'pub_year': 2012, 'pub_month': 5, 'pub_day': 18, 'stock_symbol': 'NASDAQ:FB'}
Amazon {'valuation_amount': 100000000000, 'valuation_currency_code': 'USD', 'pub_year': 1997, 'pub_month': 5, 'pub_day': None, 'stock_symbol': 'NASDAQ:AMZN'}


### 10. Retrieve the 10 companies with more employees, order by the `number of employees`

In [115]:
cur = db.companies.find().sort([('number_of_employees', DESCENDING)]).limit(10)
for e in cur:
    print(e['name'],e['number_of_employees'])

Siemens 405000
IBM 388000
Toyota 320000
PayPal 300000
Nippon Telegraph and Telephone Corporation 227000
Samsung Electronics 221726
Accenture 205000
Tata Consultancy Services 200300
Flextronics International 200000
Safeway 186000


### 11. All the companies founded on the second semester of the year. Limit your search to 1000 companies.

In [119]:
cur = db.companies.find({'$and':[{"founded_month":{"$gte":6}}, {"founded_month":{"$lte":12}}]}).limit(10)
for e in cur:
    print(e['name'],e['founded_month'])
                        

Wetpaint 10
Zoho 9
Digg 10
Omnidrive 11
Postini 6
Geni 6
Fox Interactive Media 6
eBay 9
Joost 10
Plaxo 11


### 12. All the companies founded before 2000 that have an acquisition amount of more than 10.000.00

In [125]:
cur = db.companies.find({'$and':[{"founded_year":{"$lte":2000}}, {"acquisition.price_amount":{"$gte":10000}}]}).limit(10).sort([('acquisition.price_amount',DESCENDING)]).limit(5)
for e in cur:
    print(e['name'],e['founded_year'],e['acquisition'])

BEA Systems 1995 {'price_amount': 8500000000.0, 'price_currency_code': 'USD', 'term_code': 'null', 'source_url': 'http://www.oracle.com/corporate/press/2008_jan/bea.html', 'source_description': 'Oracle to Acquire BEA Systems', 'acquired_year': 2008, 'acquired_month': 1, 'acquired_day': 16, 'acquiring_company': {'name': 'Oracle Corporation', 'permalink': 'oracle'}}
Navteq 1985 {'price_amount': 8100000000.0, 'price_currency_code': 'USD', 'term_code': 'null', 'source_url': 'http://www.businessweek.com/stories/2007-10-01/nokia-to-pay-8-dot-1-billion-for-navteqbusinessweek-business-news-stock-market-and-financial-advice', 'source_description': 'Press Release', 'acquired_year': 2008, 'acquired_month': 7, 'acquired_day': 10, 'acquiring_company': {'name': 'Nokia', 'permalink': 'nokia'}}
Sun Microsystems 1982 {'price_amount': 7400000000.0, 'price_currency_code': 'USD', 'term_code': 'cash', 'source_url': 'http://www.techcrunch.com/2009/04/20/oracle-to-buy-sun-hold-on-to-your-hats/', 'source_desc

### 13. All the companies that have been acquired after 2010, order by the acquisition amount, and retrieve only their `name` and `acquisition` field.

In [126]:
cur = db.companies.find({"acquisition.acquired_year":{"$gte":2010}}).limit(10).sort([('acquisition.price_amount',DESCENDING)]).limit(5)
for e in cur:
    print(e['name'],e['acquisition'])

T-Mobile {'price_amount': 39000000000.0, 'price_currency_code': 'USD', 'term_code': 'null', 'source_url': 'http://techcrunch.com/2011/03/20/in-the-race-for-more-spectrum-att-is-acquiring-t-mobile-for-39-billion/', 'source_description': 'In The Race For More Spectrum, AT&T Is Acquiring T-Mobile For $39 Billion', 'acquired_year': 2011, 'acquired_month': 3, 'acquired_day': 20, 'acquiring_company': {'name': 'AT&T', 'permalink': 'at-t'}}
Goodrich Corporation {'price_amount': 18400000000.0, 'price_currency_code': 'USD', 'term_code': 'null', 'source_url': 'http://www.masshightech.com/stories/2011/09/19/daily37-UTC-shells-out-184-billion-for-Goodrich.html', 'source_description': 'UTC shells out $18.4 billion for Goodrich', 'acquired_year': 2011, 'acquired_month': 9, 'acquired_day': 22, 'acquiring_company': {'name': 'United Technologies', 'permalink': 'united-technologies'}}
LSI {'price_amount': 6600000000.0, 'price_currency_code': 'USD', 'term_code': 'cash', 'source_url': 'http://dealbook.nyti

### 14. Order the companies by their `founded year`, retrieving only their `name` and `founded year`.

In [127]:
cur = db.companies.find().sort([('founded_year', DESCENDING)]).limit(5)
for e in cur:
    print(e['name'],e['founded_year'])

Fixya 2013
Wamba 2013
Advaliant 2013
Fluc 2013
iBazar 2013


### 15. All the companies that have been founded on the first seven days of the month, including the seventh. Sort them by their `acquisition price` in a descending order. Limit the search to 10 documents.

In [130]:
cur = db.companies.find({'founded_month': {'$lte': 7}}).sort([('acquisition.price_amount', DESCENDING)]).limit(10)
for e in cur:
    print(e['name'])

Sun Microsystems
Siebel Systems
Netscape
SuccessFactors
YouTube
Zappos
Meraki
Tumblr
Alibaba
Kiva Systems


### 16. All the companies on the 'web' `category` that have more than 4000 employees. Sort them by the amount of employees in ascending order.

In [135]:
cur = db.companies.find({'$and':[{'category_code': {'$eq':'web'}},{'number_of_employees':{'$gte':4000}}]}).sort([('number_of_employees',ASCENDING)]).limit(5)
for i in cur:
    print(i['name'],i['number_of_employees'])

Expedia 4400
AOL 8000
Webkinz 8657
Rakuten 10000
Los Angeles Times Media Group 10000


### 17. All the companies whose acquisition amount is more than 10.000.000, and currency is 'EUR'.

In [140]:
cur = db.companies.find({'$and':[{'acquisition.price_amount': {'$gte': 10000000}},{'acquisition.price_currency_code':{'$eq':'EUR'}}]}).limit(3)
for e in cur:
    print(e['name'],e['acquisition'])


ZYB {'price_amount': 31500000, 'price_currency_code': 'EUR', 'term_code': 'cash', 'source_url': 'http://www.techcrunch.com/2008/05/16/danish-mobile-social-network-zyb-acquired-by-vodafone-for-e315-million/', 'source_description': 'Danish Network ZYB Acquired By Vodafone For â‚¬31.5 million', 'acquired_year': 2008, 'acquired_month': 5, 'acquired_day': 16, 'acquiring_company': {'name': 'Vodafone', 'permalink': 'vodafone'}}
Apertio {'price_amount': 140000000, 'price_currency_code': 'EUR', 'term_code': 'null', 'source_url': 'http://www.techcrunch.com/2008/01/03/nokia-siemens-acquires-apertio-for-e140m/', 'source_description': 'Nokia Siemens Acquires Apertio For â‚¬140m', 'acquired_year': 2008, 'acquired_month': 1, 'acquired_day': 3, 'acquiring_company': {'name': 'Nokia Siemens Networks', 'permalink': 'nokia-siemens-networks'}}
Greenfield Online {'price_amount': 40000000, 'price_currency_code': 'EUR', 'term_code': 'null', 'source_url': 'http://www.research-live.com/news/m-and-a/toluna-buys-

### 18. All the companies that have been acquired on the first trimester of the year. Limit the search to 10 companies, and retrieve only their `name` and `acquisition` fields.

In [144]:
cur = db.companies.find({'acquisition.acquired_month':{'$lte': 3}}).limit(3)
                                                        
for e in cur:
    print(e['name'],e['acquisition'])

Kyte {'price_amount': nan, 'price_currency_code': 'USD', 'term_code': 'null', 'source_url': 'http://techcrunch.com/2011/01/31/exclusive-kit-digital-acquires-kickapps-kewego-and-kyte-for-77-2-million/', 'source_description': 'KIT digital Acquires KickApps, Kewego AND Kyte For $77.2 Million', 'acquired_year': 2011, 'acquired_month': 1, 'acquired_day': 31, 'acquiring_company': {'name': 'KIT digital', 'permalink': 'kit-digital'}}
NetRatings {'price_amount': 327000000, 'price_currency_code': 'USD', 'term_code': 'cash', 'source_url': 'http://login.vnuemedia.com/hr/login/login_subscribe.jsp?id=0oqDem1gYIfIclz9i2%2Ffqj5NxCp2AC5DPbVnyT2da8GyV2mXjasabE128n69OrmcAh52%2FGE3pSG%2F%0AEKRYD9vh9EhrJrxukmUzh532fSMTZXL42gwPB80UWVtF1NwJ5UZSM%2BCkLU1mpYBoHFgiH%2Fi0f6Ax%0A9yMIVxt47t%2BHamhEQ0nkOEK24L', 'source_description': 'Nielsen buys rest of NetRatings', 'acquired_year': 2007, 'acquired_month': 2, 'acquired_day': nan, 'acquiring_company': {'name': 'Nielsen', 'permalink': 'nielsen'}}
blogTV {'price_amou

# Bonus
### 19. All the companies that have been founded between 2000 and 2010, but have not been acquired before 2011.

In [154]:
#no lo he conseguido :(
year = list(range(2000,2011))
cur = db.companies.find({'$and'[{'founded_year': {'$eq': year}},{'acquisition.acquired_year':{'$not':{'$lte': 2011}}}]})
for i in cur:
    print(i['name'])

TypeError: string indices must be integers

### 20. All the companies that have been 'deadpooled' after the third year.

In [160]:
#tampoco lo he conseguido
cur = db.companies.find({'$and'[{'deadpool_year':{'$exists':'true'}},{'deadpool_year':{'$gte': ['founded_year'+3] }}]})

TypeError: can only concatenate str (not "int") to str